# Training

## Setup

In [1]:
import torch
import torchvision
from torchvision import datasets, transforms
import pandas as pd
from torch.utils.data import random_split
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
#from torch.autograd import Variable

In [2]:
if torch.cuda.is_available():
    device = torch.device('cuda')
elif torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')
#torch.backends.cudnn.enabled = False
val_size = 5000
test_size = 5000
batch_size = 128
num_workers = 4
pin_memory = False if device == torch.device('cpu') else True

transform = transforms.Compose([transforms.ToTensor()])

## Potential changes - Markus

# Normalize input images to [-1, 1]
# transform = transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Normalize(mean=(0.5,), std=(0.5,))
# ])


# Downloading MNIST again :) Training (60k) and test(5k) + val(5k) split
train_loader = torch.utils.data.DataLoader(datasets.MNIST('./mnist_data',
                                            download=True,
                                            train=True,
                                            transform=transform),
                                            batch_size=batch_size,
                                            shuffle=True, num_workers=num_workers,
                                            pin_memory=pin_memory,
                                            drop_last=True)

test_dataset = datasets.MNIST('./mnist_data',
                               download=True,
                               train=False,
                               transform=transform)

val_dataset, test_dataset = random_split(test_dataset, [val_size, test_size])

# Test set to compare with DDPM paper
test_loader = torch.utils.data.DataLoader(test_dataset,
                                            batch_size=batch_size,
                                            shuffle=False, num_workers=num_workers,
                                            pin_memory=pin_memory)

# Validation set so we can keep track of approximated FID score while training
validation_loader = torch.utils.data.DataLoader(val_dataset,
                                            batch_size=batch_size,
                                            shuffle=False, num_workers=num_workers,
                                            pin_memory=pin_memory)
    

In [3]:
# # Sets up alpha_bar for training and test so alpha_bar_t = alpha_bar[t]
# T = 1000
# beta_start, beta_end = [1e-4, 2e-02]
# beta = torch.linspace(beta_start, beta_end, T)
# alpha = 1-beta
# alpha_bar = alpha.clone()
# for e in range(T-1):
#     alpha_bar[e+1] *= alpha_bar[e]

# alpha = alpha.view((T, 1, 1, 1)).to(device)
# beta = beta.view((T, 1, 1, 1)).to(device)
# alpha_bar = alpha_bar.view((T, 1, 1, 1)).to(device)


### Potential changes - Markus
T = 1000
beta_start, beta_end = 1e-4, 2e-2
beta = torch.linspace(beta_start, beta_end, T)  # Linear noise schedule
alpha = 1.0 - beta
alpha_bar = torch.cumprod(alpha, dim=0)  # Cumulative product for alpha_bar

# Reshape for broadcasting (if required for your model)
alpha = alpha.view((T, 1, 1, 1)).to(device)
beta = beta.view((T, 1, 1, 1)).to(device)
alpha_bar = alpha_bar.view((T, 1, 1, 1)).to(device)


## Model

In [4]:
class UNET(torch.nn.Module):
    def __init__(self):
        super(UNET, self).__init__()
        channels = [32, 64, 128, 256]
        self.convs = nn.ModuleList([
            nn.Sequential(
                nn.Conv2d(2, channels[0], kernel_size=3, padding=1),  # (batchsize, 32, 28, 28)
                nn.ReLU(),
            ),
            nn.Sequential(
                nn.MaxPool2d(2),  # (batchsize, 32, 14, 14)
                nn.Conv2d(channels[0], channels[1], kernel_size=3, padding=1),  # (batchsize, 64, 14, 14)
                nn.ReLU(),
            ),
            nn.Sequential(
                nn.MaxPool2d(2),  # (batchsize, 64, 7, 7)
                nn.Conv2d(channels[1], channels[2], kernel_size=3, padding=1),  # (batchsize, 128, 7, 7)
                nn.ReLU(),
            ),
            nn.Sequential(
                nn.MaxPool2d(2, padding=1),  # (batchsize, 128, 4, 4)
                nn.Conv2d(channels[2], channels[3], kernel_size=3, padding=1),  # (batchsize, 256, 4, 4)
                nn.ReLU(),
            )
        ])

        self.tconvs = nn.ModuleList([
            nn.Sequential(
                nn.ConvTranspose2d(channels[3], channels[2], kernel_size=3, 
                                   stride=2, padding=1, output_padding=0),   # (batchsize, 128, 7, 7)
                nn.ReLU()
            ),
            nn.Sequential(
                nn.ConvTranspose2d(channels[2]*2, channels[1], kernel_size=3,
                                   stride=2, padding=1, output_padding=1),   # (batchsize, 64, 14, 14)
                nn.ReLU()
            ),
            nn.Sequential(
                nn.ConvTranspose2d(channels[1]*2, channels[0], kernel_size=3, 
                                   stride=2, padding=1, output_padding=1),   # (batchsize, 32, 28, 28)
                nn.ReLU()
            ),
            nn.Sequential(
                nn.Conv2d(channels[0]*2,channels[0],kernel_size=3,padding=1),  # (batchsize, 32, 28, 28)
                nn.ReLU(),
                nn.Conv2d(channels[0],1,kernel_size=1) # (batchsize, 1, 28, 28)
            )      
        ])

    def forward(self, x, t):
        x_trans = torch.cat((x, t), dim=-3)
        signal = x_trans
        signals = []

        for i, conv in enumerate(self.convs):
            # print(f"conv {i}")
            signal = conv(signal)
            # print(signal.shape)
            if i < len(self.convs)-1:
                signals.append(signal)
                
        for i, tconv in enumerate(self.tconvs):
            # print(f"tconv {i}")
            # print(f"signal shape: {signal.shape}")
            if i == 0:
                signal = tconv(signal)
            else:
                signal = torch.cat((signal, signals[-i]), dim=-3)
                signal = tconv(signal)
        return signal

## Training loop

In [5]:
#from UNET import UNET
epochs = 200
model = UNET()
model.to(device)
model.train()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
criterion = torch.nn.MSELoss()
running_loss = 0

In [6]:
for epoch in range(epochs):
    for e, data in enumerate(train_loader):
        x0, _ = data
        x0 = x0.to(device)
        t = torch.randint(1, T+1, (batch_size,)).to(device)
        eps = torch.randn(batch_size, 1, 28, 28).to(device)
        # print(eps.shape)
        # print(x0.shape)
        xt = torch.sqrt(alpha_bar[t-1]) * x0 + torch.sqrt(1 - alpha_bar[t-1]) * eps
        loss = criterion(eps, model(xt, t.view(batch_size, 1, 1, 1).expand(batch_size, 1, 28, 28)))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

        if e % 100 == 99:
            print(f'{epoch, e+1}, Average_loss: {running_loss/100:.3f}')
            running_loss = 0.0
    if epoch % 10 == 9:
        torch.save(model.state_dict(), f"DDPM_{epoch+1}.pth")

(0, 100), Average_loss: 775.290
(0, 200), Average_loss: 2.717
(0, 300), Average_loss: 1.126
(0, 400), Average_loss: 0.998
(1, 100), Average_loss: 1.623
(1, 200), Average_loss: 0.947
(1, 300), Average_loss: 0.934
(1, 400), Average_loss: 0.921
(2, 100), Average_loss: 1.513
(2, 200), Average_loss: 0.881
(2, 300), Average_loss: 0.865
(2, 400), Average_loss: 0.848
(3, 100), Average_loss: 1.383
(3, 200), Average_loss: 0.797
(3, 300), Average_loss: 0.775
(3, 400), Average_loss: 0.752
(4, 100), Average_loss: 1.217
(4, 200), Average_loss: 0.694
(4, 300), Average_loss: 0.674
(4, 400), Average_loss: 0.651
(5, 100), Average_loss: 1.051
(5, 200), Average_loss: 0.595
(5, 300), Average_loss: 0.576
(5, 400), Average_loss: 0.551
(6, 100), Average_loss: 0.889
(6, 200), Average_loss: 0.500
(6, 300), Average_loss: 0.480
(6, 400), Average_loss: 0.464
(7, 100), Average_loss: 0.745
(7, 200), Average_loss: 0.369
(7, 300), Average_loss: 0.331
(7, 400), Average_loss: 0.298
(8, 100), Average_loss: 0.439
(8, 200)

KeyboardInterrupt: 

# Sampling

In [ ]:
with torch.inference_mode():
    model.eval()
    batch_size = 1
    xt = torch.randn(batch_size, 1, 28, 28).to(device)

    for t in torch.arange(T, 0, -1):
        print(t)
        t = t.to(device)
        z = torch.randn(batch_size, 1, 28, 28).to(device) if t > 1 else torch.zeros(batch_size, 1, 28, 28).to(device)
        xt_new = 1 / torch.sqrt(alpha[t - 1]) * (xt - (1 - alpha[t - 1])/(torch.sqrt(1 - alpha_bar[t - 1])) * 
                                                    model(xt, t.view(batch_size, 1, 1, 1).expand(batch_size, 1, 28, 28))) + torch.sqrt(beta[t-1]) * z
        xt = xt_new
        

In [ ]:
plt.imshow(xt[0][0].cpu().detach().numpy(), cmap="grey")

In [ ]:
# sample from all the timesteps
fig, axs = plt.subplots(1, 20, tight_layout=True, figsize=(40, 20))
tens = np.arange(10,201,10)
seed = 3546
for ten in tens:
    with torch.inference_mode():
        # torch.manual_seed(seed)
        model.load_state_dict(torch.load(f"DATA_FROM_training/DDPM_{ten}.pth", map_location=device))
        model.eval()
        batch_size = 1
        xt = torch.randn(batch_size, 1, 28, 28).to(device)

        for t in torch.arange(T, 0, -1):
            t = t.to(device)
            z = torch.randn(batch_size, 1, 28, 28).to(device) if t > 1 else torch.zeros(batch_size, 1, 28, 28).to(device)
            xt_new = 1 / torch.sqrt(alpha[t - 1]) * (xt - (1 - alpha[t - 1])/(torch.sqrt(1 - alpha_bar[t - 1])) * 
                                                        model(xt, t.view(batch_size, 1, 1, 1).expand(batch_size, 1, 28, 28))) + torch.sqrt(beta[t-1]) * z
            xt = xt_new
            axs[(ten//10)-1].imshow(xt[0][0].cpu().detach().numpy(), cmap="gray")
            axs[(ten//10)-1].axis("off")
            axs[(ten//10)-1].set_title(f"Epoch={ten}")
plt.show()